In [17]:
from client import Client
import requests
import json
import numpy as np
from sklearn.metrics import log_loss

client = Client("node1","0.0.0.0")

In [18]:
dataset = client.load_data("../datasets/UNSW_NB15_training-set.csv")

In [19]:
x, y = client.preprocess_data(dataset)

In [20]:
y_test = y[60000:]
x_test = x[60000:]

In [21]:
x = x[:30000]
y = y[:30000]

In [22]:
x.shape

(30000, 189)

In [23]:
coef, intercept, acc, clf = client.train_model(x,y)

/home/tk/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [26]:
clf.predict_proba(x_test)

array([[0.34306016, 0.65693984],
       [0.07415381, 0.92584619],
       [0.06338353, 0.93661647],
       ...,
       [0.50037431, 0.49962569],
       [0.5003743 , 0.4996257 ],
       [0.12893972, 0.87106028]])

In [30]:
loss = log_loss(y_test,clf.predict(x_test), eps=1e-15)

In [31]:
loss

24.283792278940282

In [34]:
client.test_model(clf,y_test,x_test)

0.29692817481640693

In [52]:
headers = {'content-type': 'application/json'}
url = 'http://127.0.0.1:5000/register'

data = {
    "username" : "Tom",
    "email" : "tom@test.com",
    "password" : "123456789"
}

reponse = requests.post(url, data=json.dumps(data), headers=headers)

In [53]:
reponse.text

'{"id": 1}'

In [54]:
headers = {'content-type': 'application/json'}
url = 'http://127.0.0.1:5000/local_model'

data = {
    "intercept" : intercept[0],
    "bias" : coef.tolist()[0],
    "author" : 1
}

response = requests.post(url, data=json.dumps(data), headers=headers)
response.text

'{"result": "model added"}'

### After global model calculation

In [109]:
headers = {'content-type': 'application/json'}
url = 'http://127.0.0.1:5000/model/global'

response = requests.get(url, headers=headers)
json = response.json()
coef_global = json['bias']
inter_global = json['intercept']

In [110]:
clf.intercept_ = np.array([inter_global])
clf.coef_ = np.array([coef_global])

In [111]:
client.test_model(clf,y_test,x_test)

0.2547465520329572